In [1]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import logging
import requests
import os
import nltk
import cmudict
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd




In [2]:
ip=pd.read_excel('Input.xlsx')
print(ip)
url_list=ip['URL']
type(url_list)



for i in range(len(ip)):
    url_id = ip.at[i, 'URL_ID']
    
# Check if the file exists

    if os.path.exists(f'{str(url_id)}.txt'):
    # Delete the file
        os.remove(f'{str(url_id)}.txt')
        print(f"The file '{str(url_id)}.txt' has been deleted.")
    

      URL_ID                                                URL
0      123.0  https://insights.blackcoffer.com/rise-of-telem...
1      321.0  https://insights.blackcoffer.com/rise-of-e-hea...
2     2345.0  https://insights.blackcoffer.com/rise-of-e-hea...
3     4321.0  https://insights.blackcoffer.com/rise-of-telem...
4      432.0  https://insights.blackcoffer.com/rise-of-telem...
..       ...                                                ...
109  50921.0  https://insights.blackcoffer.com/coronavirus-i...
110  51382.8  https://insights.blackcoffer.com/coronavirus-i...
111  51844.6  https://insights.blackcoffer.com/what-are-the-...
112  52306.4  https://insights.blackcoffer.com/marketing-dri...
113  52768.2  https://insights.blackcoffer.com/continued-dem...

[114 rows x 2 columns]
The file '123.0.txt' has been deleted.
The file '321.0.txt' has been deleted.
The file '2345.0.txt' has been deleted.
The file '4321.0.txt' has been deleted.
The file '432.0.txt' has been deleted.
The file '2

In [3]:
article_title=[]
article_text=[]
article_data={}
df=pd.DataFrame()


for index in range(len(ip)):
    url_id = ip.at[index, 'URL_ID']
    url = ip.at[index, 'URL']
    html_text = requests.get(url).text
    title_html = bs(html_text, 'html5lib')
    title = title_html.find("h1", class_=["entry-title","tdb-title-text"]).string if title_html.find("h1", class_=["entry-title","tdb-title-text"])!=None else f'h1 tag not available for {url}'
    content = title_html.find_all("div", class_=["tdb-block-inner td-fix-index", "td-pb-span8 td-main-content","td-post-content tagdiv-type"])
    article_text = []
    for div in content:
        inner_div = div.find("div")  # Assuming you want to find a single nested div
        if inner_div:
            paragraphs = inner_div.find_all("p")
            for p in paragraphs:
                article_text.append(p.get_text())
    concatenated_text = ' '.join(article_text)
    article_data[title]=title
    article_data[url] = {'url': url, 'text': concatenated_text}    
    content = title_html.find_all("div", class_=["tdb-block-inner td-fix-index", "td-pb-span8 td-main-content","td-post-content tagdiv-type"])
    for div in content:
        paragraphs = div.find_all("p")
        for p in paragraphs:
            article_text.append(p.get_text())
    concatenated_text = ' '.join(article_text)
    article_data[title]=concatenated_text
    #article_data[url] = {'title': title, 'text': concatenated_text}
    
    df=pd.concat([df,pd.DataFrame([[url,concatenated_text]], columns=['url', 'text'])],axis=0,ignore_index=True)
    df.to_csv(f'{str(url_id)}.txt', index=False, sep='\t')
    
    
    

    



    

In [4]:
df

,url,text
0,https://insights.blackcoffer.com/rise-of-telem...,"Telemedicine, the use of technology to diagnos..."
1,https://insights.blackcoffer.com/rise-of-e-hea...,"The rise of e-health, or the use of electronic..."
2,https://insights.blackcoffer.com/rise-of-e-hea...,2020 was the year the world was ravaged by the...
3,https://insights.blackcoffer.com/rise-of-telem...,"“More gains on quality, affordability and acce..."
4,https://insights.blackcoffer.com/rise-of-telem...,"“More gains on quality, affordability and acce..."
...,...,...
109,https://insights.blackcoffer.com/coronavirus-i...,Before jumping on the topic I would like to gi...
110,https://insights.blackcoffer.com/coronavirus-i...,As the coronavirus spreads around the world an...
111,https://insights.blackcoffer.com/what-are-the-...,"From Alibaba to Ping An and Google to Ford, co..."
112,https://insights.blackcoffer.com/marketing-dri...,"When\nthe British ruled India, many Indians\na..."


In [5]:
def get_stop_words():
    stop_words = []
    files = os.listdir('./Sentiment Analysis')
    
    for file in files:
        if file[0:4] == 'Stop':
            with open(f'./Sentiment Analysis/{file}', encoding='latin-1') as f:
                words = f.read()

            #stop_words = words.split('\n')
            stop_words = word_tokenize(words)
            
    return stop_words

# print(get_stop_words())

In [6]:
with open('./Sentiment Analysis/positive-words.txt', encoding='latin-1') as f:
    words = f.read()
positive_words = word_tokenize(words)

with open('./Sentiment Analysis/negative-words.txt', encoding='latin-1') as f:
    words = f.read()
    
negative_words =  word_tokenize(words)

stop_words = get_stop_words()

stops = list(set(stopwords.words('english')))

# print(stops)

In [7]:
word_dict = {'positive': [], 'negative': []}
for word in positive_words:
    if word not in stop_words:
        word_dict['positive'].append(word)
        
for word in negative_words:
    if word not in stop_words:
        word_dict['negative'].append(word)

In [8]:
clean_words_list = []
word_count=[]

for text in df['text']:
    words = word_tokenize(text)
    clean_words_row = [word for word in words if word not in stops and word not in ['?', '!', ',', '.', ':',';','/','-', '#', '“', '’', ')','&','(', '”', '%', '–','|','[', ']']]
    clean_words_list.append(clean_words_row)
    word_count.append(len(clean_words_row))
            

print(word_count)

[3126, 1032, 676, 2340, 2340, 2055, 1983, 2955, 270, 1878, 675, 2382, 1011, 1896, 1692, 2193, 2004, 1170, 2373, 3102, 2775, 3003, 2064, 1269, 0, 1230, 3723, 3384, 2127, 2331, 2598, 914, 933, 2769, 1338, 1278, 318, 0, 2421, 1254, 3105, 1164, 681, 1440, 861, 609, 285, 1323, 312, 1272, 237, 576, 3012, 2808, 2556, 1272, 2106, 717, 2892, 1638, 1803, 1809, 1968, 1686, 1146, 483, 1320, 2145, 7431, 2913, 2829, 76, 488, 2988, 3006, 1881, 3201, 1668, 1989, 1866, 827, 309, 1905, 1167, 2034, 1608, 285, 332, 527, 1068, 1935, 1746, 622, 1419, 2640, 2955, 1443, 3111, 2025, 627, 4164, 2391, 3096, 3225, 1800, 3081, 1449, 2370, 486, 918, 3357, 3090, 2451, 1272]


In [9]:
clean_words_list

[['Telemedicine',
  'use',
  'technology',
  'diagnose',
  'treat',
  'patients',
  'remotely',
  'rising',
  'recent',
  'years',
  'With',
  'advent',
  'high-speed',
  'internet',
  'improved',
  'video',
  'conferencing',
  'tools',
  'healthcare',
  'providers',
  'increasingly',
  'turning',
  'telemedicine',
  'provide',
  'care',
  'patients',
  'remote',
  'underserved',
  'areas',
  'Telemedicine',
  'using',
  'technology',
  'provide',
  'healthcare',
  'services',
  'remotely',
  'recently',
  'gained',
  'popularity',
  'With',
  'advancements',
  'communication',
  'medical',
  'technology',
  'become',
  'increasingly',
  'possible',
  'doctors',
  'patients',
  'connect',
  'interact',
  'anywhere',
  'world',
  'This',
  'led',
  'rise',
  'telemedicine',
  'potential',
  'revolutionize',
  'way',
  'healthcare',
  'delivered.',
  'Telemedicine',
  'The',
  'increasing',
  'focus',
  'preventative',
  'healthcare',
  'also',
  'driven',
  'rise',
  'telemedicine',
  '

In [10]:
positive_score_list=[]
negative_score_list=[]
polarity_score_list=[]
subjectivity_score_list=[]
def calculate_positive_score(clean_words):
    for text in clean_words_list:
        positive_score = 0
        for positive in word_dict['positive']:
            if positive in text:
                positive_score+=1
        positive_score_list.append(positive_score)
    return positive_score_list
def calculate_negative_score(clean_words):
    for text in clean_words_list:
        negative_score = 0
        for negative in word_dict['negative']:
            if negative in text:
                negative_score+=1
        negative_score_list.append(negative_score)
    return negative_score_list
def calculate_polarity_score(positive_score,negative_score):
    for i in range(0,len(url_list)):
        polarity_score = 0
        polarity_score = (positive_score[i] - negative_score[i])/ ((positive_score[i] + negative_score[i]) + 0.000001)
        polarity_score=round(polarity_score,2)
        polarity_score_list.append(polarity_score)
    return polarity_score_list
def calculate_subjectivity_score(positive_score,negative_score,clean_words_list):
    for i in range(0,len(url_list)):
        subjectivity_score=0
        subjectivity_score = (positive_score[i] + negative_score[i])/ (len(text[i]) + 0.000001)
        subjectivity_score=round(subjectivity_score,2)
        subjectivity_score_list.append(subjectivity_score)
    return subjectivity_score_list


In [11]:
positive_score_list = calculate_positive_score(clean_words_list)
print("Positive Score List:", positive_score_list)

Positive Score List: [36, 22, 16, 33, 33, 33, 29, 46, 8, 21, 14, 29, 21, 15, 26, 24, 25, 47, 41, 51, 41, 39, 37, 19, 0, 27, 52, 36, 35, 22, 43, 51, 23, 53, 20, 14, 1, 0, 22, 16, 26, 37, 6, 20, 12, 15, 6, 23, 9, 24, 2, 9, 22, 33, 33, 20, 29, 24, 31, 23, 24, 12, 19, 23, 19, 8, 15, 20, 12, 32, 22, 5, 32, 23, 37, 20, 15, 17, 32, 20, 17, 6, 27, 7, 23, 18, 1, 13, 24, 25, 7, 14, 23, 18, 30, 13, 10, 34, 25, 5, 29, 35, 32, 39, 24, 52, 15, 31, 6, 4, 18, 62, 20, 19]


In [12]:
negative_score_list = calculate_negative_score(clean_words_list)
print("Negative Score List:", negative_score_list)

Negative Score List: [17, 11, 23, 25, 25, 9, 9, 6, 0, 32, 8, 13, 3, 9, 14, 17, 22, 25, 29, 25, 19, 21, 21, 10, 0, 13, 30, 35, 13, 18, 19, 15, 0, 28, 0, 6, 3, 0, 40, 9, 14, 3, 1, 10, 7, 8, 0, 23, 2, 10, 4, 21, 37, 20, 24, 7, 7, 5, 12, 46, 17, 18, 19, 51, 8, 6, 15, 28, 16, 37, 38, 1, 4, 30, 13, 22, 25, 28, 24, 18, 29, 3, 40, 23, 37, 26, 0, 3, 41, 17, 35, 22, 47, 20, 30, 31, 15, 41, 34, 9, 45, 44, 48, 56, 29, 17, 19, 55, 6, 13, 40, 25, 16, 18]


In [13]:
positive_score_list = calculate_positive_score(clean_words_list)
negative_score_list = calculate_negative_score(clean_words_list)
polarity_score_list = calculate_polarity_score(positive_score_list, negative_score_list)
subjectivity_score_list = calculate_subjectivity_score(positive_score_list, negative_score_list, clean_words_list)

# Print the results
print("Positive Score List:", positive_score_list)
print("Negative Score List:", negative_score_list)
print("Polarity Score List:", polarity_score_list)
print("Subjectivity Score List:", subjectivity_score_list)

Positive Score List: [36, 22, 16, 33, 33, 33, 29, 46, 8, 21, 14, 29, 21, 15, 26, 24, 25, 47, 41, 51, 41, 39, 37, 19, 0, 27, 52, 36, 35, 22, 43, 51, 23, 53, 20, 14, 1, 0, 22, 16, 26, 37, 6, 20, 12, 15, 6, 23, 9, 24, 2, 9, 22, 33, 33, 20, 29, 24, 31, 23, 24, 12, 19, 23, 19, 8, 15, 20, 12, 32, 22, 5, 32, 23, 37, 20, 15, 17, 32, 20, 17, 6, 27, 7, 23, 18, 1, 13, 24, 25, 7, 14, 23, 18, 30, 13, 10, 34, 25, 5, 29, 35, 32, 39, 24, 52, 15, 31, 6, 4, 18, 62, 20, 19, 36, 22, 16, 33, 33, 33, 29, 46, 8, 21, 14, 29, 21, 15, 26, 24, 25, 47, 41, 51, 41, 39, 37, 19, 0, 27, 52, 36, 35, 22, 43, 51, 23, 53, 20, 14, 1, 0, 22, 16, 26, 37, 6, 20, 12, 15, 6, 23, 9, 24, 2, 9, 22, 33, 33, 20, 29, 24, 31, 23, 24, 12, 19, 23, 19, 8, 15, 20, 12, 32, 22, 5, 32, 23, 37, 20, 15, 17, 32, 20, 17, 6, 27, 7, 23, 18, 1, 13, 24, 25, 7, 14, 23, 18, 30, 13, 10, 34, 25, 5, 29, 35, 32, 39, 24, 52, 15, 31, 6, 4, 18, 62, 20, 19]
Negative Score List: [17, 11, 23, 25, 25, 9, 9, 6, 0, 32, 8, 13, 3, 9, 14, 17, 22, 25, 29, 25, 19, 21,

In [14]:
num_sentences_list = []

for text in df['text']:
    num_sentences = text.count('.')
    num_sentences_list.append(num_sentences)

# Now num_sentences_list contains the number of sentences for each row
print(len(num_sentences_list))


114


In [15]:
print(len(calculate_positive_score(clean_words_list)))

342


In [16]:
avg_num_words_per_sentences = []

for text in df['text']:
    num_sentences = text.count('.')
    num_words = len(text.split(' '))
    
    try:
        avg_num_words_sentences = num_words / num_sentences
        avg_num_words_sentences=round(avg_num_words_sentences,2)
        avg_num_words_per_sentences.append(avg_num_words_sentences)
    except ZeroDivisionError:
        # If num of sentences is zero, append 0 or any other default value
        avg_num_words_per_sentences.append(0)

print(avg_num_words_per_sentences)

[19.69, 25.0, 14.63, 20.91, 20.91, 16.95, 21.35, 17.77, 21.54, 20.24, 15.15, 21.65, 23.74, 26.23, 18.17, 24.33, 24.38, 23.33, 20.33, 20.05, 19.38, 21.12, 22.8, 16.16, 0, 19.94, 25.27, 18.57, 21.69, 16.78, 21.95, 19.12, 23.48, 256.86, 17.02, 17.52, 28.0, 0, 24.05, 23.0, 20.98, 15.05, 22.81, 23.35, 23.35, 13.96, 17.0, 14.8, 19.56, 17.21, 20.71, 19.74, 23.41, 24.81, 21.25, 12.97, 24.58, 28.81, 23.6, 14.07, 15.51, 22.07, 16.0, 15.34, 20.03, 28.6, 31.83, 21.0, 13.95, 21.09, 24.0, 23.0, 22.89, 15.49, 19.53, 22.83, 23.29, 19.75, 16.89, 30.28, 19.25, 22.38, 19.16, 18.73, 21.78, 28.11, 34.5, 18.45, 28.59, 23.93, 21.02, 20.04, 20.14, 23.21, 21.22, 21.07, 36.21, 25.44, 16.3, 17.32, 22.46, 25.33, 16.93, 22.42, 18.17, 29.85, 26.87, 21.73, 20.43, 19.42, 32.4, 23.35, 20.13, 23.73]


In [17]:
def count_syllables(word):
    vowels = ['a', 'e', 'i', 'o', 'u']
    syl_count = 0
    
    for i in range(len(word)):
        if word[i] in vowels:
            syl_count += 1
            
    if word.endswith(('ed', 'es')):
        syl_count -= 1
    
    return syl_count

def count_syllables_in_text(text):
    words = word_tokenize(text)
    syllable_count = sum(count_syllables(word) for word in words)
    complex_count=0
    for word in words:
        if count_syllables(word)>2:
            complex_count+=1
            
    return [complex_count,syllable_count]

# Assuming df is your DataFrame and 'text' is a column in df
print(count_syllables_in_text(df.at[0, 'text']))


[1644, 10689]


In [18]:
#Percentage of complex words
perc_complex_words_list=[]
complex_count_list=[]
syl_count_list=[]
for text in df['text']:
    num_words = len(text.split(' '))
    [complex_count,syl_count]=count_syllables_in_text(text)
    perc_complex_words=complex_count/num_words
    perc_complex_words_list.append(perc_complex_words)
    complex_count_list.append(complex_count)
    syl_count_list.append(syl_count)
    

In [19]:
fog_index_list=[]
for i in range(len(df)):
    fog_index=0.4*(avg_num_words_per_sentences[i]+perc_complex_words_list[i])
    fog_index_list.append(fog_index)
    

In [20]:
negative_score_list

[17,
 11,
 23,
 25,
 25,
 9,
 9,
 6,
 0,
 32,
 8,
 13,
 3,
 9,
 14,
 17,
 22,
 25,
 29,
 25,
 19,
 21,
 21,
 10,
 0,
 13,
 30,
 35,
 13,
 18,
 19,
 15,
 0,
 28,
 0,
 6,
 3,
 0,
 40,
 9,
 14,
 3,
 1,
 10,
 7,
 8,
 0,
 23,
 2,
 10,
 4,
 21,
 37,
 20,
 24,
 7,
 7,
 5,
 12,
 46,
 17,
 18,
 19,
 51,
 8,
 6,
 15,
 28,
 16,
 37,
 38,
 1,
 4,
 30,
 13,
 22,
 25,
 28,
 24,
 18,
 29,
 3,
 40,
 23,
 37,
 26,
 0,
 3,
 41,
 17,
 35,
 22,
 47,
 20,
 30,
 31,
 15,
 41,
 34,
 9,
 45,
 44,
 48,
 56,
 29,
 17,
 19,
 55,
 6,
 13,
 40,
 25,
 16,
 18,
 17,
 11,
 23,
 25,
 25,
 9,
 9,
 6,
 0,
 32,
 8,
 13,
 3,
 9,
 14,
 17,
 22,
 25,
 29,
 25,
 19,
 21,
 21,
 10,
 0,
 13,
 30,
 35,
 13,
 18,
 19,
 15,
 0,
 28,
 0,
 6,
 3,
 0,
 40,
 9,
 14,
 3,
 1,
 10,
 7,
 8,
 0,
 23,
 2,
 10,
 4,
 21,
 37,
 20,
 24,
 7,
 7,
 5,
 12,
 46,
 17,
 18,
 19,
 51,
 8,
 6,
 15,
 28,
 16,
 37,
 38,
 1,
 4,
 30,
 13,
 22,
 25,
 28,
 24,
 18,
 29,
 3,
 40,
 23,
 37,
 26,
 0,
 3,
 41,
 17,
 35,
 22,
 47,
 20,
 30,
 31,
 15,
 41,
 34,

In [21]:
pronoun_count_list=[]
import re

def count_personal_pronouns(text):
    # Define a regex pattern for personal pronouns
    pronoun_pattern = re.compile(r'\b(I|we|my|ours|us|We|My|Ours|Us|Them|them|they|They|our|Our)\b')

    # Find all matches in the text
    matches = re.findall(pronoun_pattern, text)
    return len(matches)
    

for text in df['text']:
    pronoun_count = count_personal_pronouns(text)
    pronoun_count_list.append(pronoun_count)
print(pronoun_count_list)

[45, 15, 7, 54, 54, 24, 18, 96, 10, 48, 27, 36, 12, 27, 21, 117, 51, 9, 42, 27, 84, 99, 153, 45, 0, 18, 45, 36, 54, 21, 174, 21, 0, 108, 18, 6, 12, 0, 18, 24, 48, 12, 6, 15, 15, 24, 9, 180, 18, 18, 18, 33, 78, 96, 114, 18, 57, 45, 30, 75, 33, 24, 45, 81, 39, 12, 15, 54, 0, 69, 0, 4, 11, 60, 54, 63, 9, 27, 153, 51, 3, 12, 78, 24, 15, 69, 3, 10, 7, 150, 0, 15, 3, 36, 18, 15, 147, 42, 36, 3, 30, 171, 51, 54, 45, 42, 24, 30, 27, 3, 6, 24, 33, 15]


In [22]:
def word_length(text):
    t = text.split(' ')
    word_count = len(t)
    tot_char = 0  # Initialize tot_char to 0

    for t1 in t:
        tot_char += len(t1)  # Accumulate the length of each word

    if word_count > 0:
        average_word_length = tot_char / word_count
        return round(average_word_length,2)
    else:
        return 0

# Assuming df is a DataFrame and 'text' is a column in df
# Initialize avg_word_length as an empty list
avg_word_length = []

for text in df['text']:
    word_length_value = word_length(text)
    word_length_value=round(word_length_value,2)
    avg_word_length.append(word_length_value)


# Assuming ip, positive_score_list, negative_score_list, polarity_score_list, subjectivity_score_list,
# avg_num_words_per_sentences, word_count, pronoun_count_list are already defined

# Concatenate the Series and other columns into a DataFrame

        
        

In [23]:
positive_score_list=pd.Series(positive_score_list)
negative_score_list=pd.Series(negative_score_list)
polarity_score_list=pd.Series(polarity_score_list)
subjectivity_score_list=pd.Series(subjectivity_score_list)
avg_num_words_per_sentences=pd.Series(avg_num_words_per_sentences)
word_count=pd.Series(word_count)
pronoun_count_list=pd.Series(pronoun_count_list)
avg_word_length=pd.Series(avg_word_length)
fog_index_list=pd.Series(fog_index_list)
perc_complex_words_list=pd.Series(perc_complex_words_list)
complex_count_list=pd.Series(complex_count_list)
syl_count_list=pd.Series(syl_count_list)

In [24]:
op_sentiment = pd.concat([ip, positive_score_list, negative_score_list, polarity_score_list,
                          subjectivity_score_list, avg_num_words_per_sentences,perc_complex_words_list,fog_index_list,
                          avg_num_words_per_sentences,complex_count_list, word_count,syl_count_list, pronoun_count_list,
                          avg_word_length], axis=1)

# Rename the columns
op_sentiment.columns = ['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE',
                        'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX',
                        'AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT', 'WORD COUNT','SYLLABLE PER WORD',
                        'PERSONAL_PRONOUNS', 'AVG WORD LENGTH']


In [25]:
op_sentiment[0:10]

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL_PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,36,17.0,0.36,53.0,19.69,0.331318,8.008527,19.69,1644.0,3126.0,10689.0,45.0,5.71
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,22,11.0,0.33,33.0,25.00,0.325000,10.130000,25.00,585.0,1032.0,3750.0,15.0,5.53
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,16,23.0,-0.18,39.0,14.63,0.267790,5.959116,14.63,286.0,676.0,2017.0,7.0,5.43
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,33,25.0,0.14,58.0,20.91,0.300082,8.484033,20.91,1092.0,2340.0,7311.0,54.0,5.65
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,33,25.0,0.14,58.0,20.91,0.300082,8.484033,20.91,1092.0,2340.0,7311.0,54.0,5.65
5,2893.8,https://insights.blackcoffer.com/rise-of-chatb...,33,9.0,0.57,42.0,16.95,0.287558,6.895023,16.95,936.0,2055.0,6417.0,24.0,5.63
6,3355.6,https://insights.blackcoffer.com/rise-of-e-hea...,29,9.0,0.53,38.0,21.35,0.322180,8.668872,21.35,1011.0,1983.0,6411.0,18.0,5.73
7,3817.4,https://insights.blackcoffer.com/how-does-mark...,46,6.0,0.77,52.0,17.77,0.277251,7.218900,17.77,1404.0,2955.0,9729.0,96.0,5.42
8,4279.2,https://insights.blackcoffer.com/how-advertise...,8,0.0,1.00,8.0,21.54,0.216634,8.702654,21.54,112.0,270.0,919.0,10.0,5.08
9,4741.0,https://insights.blackcoffer.com/negative-effe...,21,32.0,-0.21,53.0,20.24,0.332016,8.228806,20.24,1008.0,1878.0,6306.0,48.0,5.72


In [26]:
len(pronoun_count_list)

114

In [27]:
op_sentiment.to_excel('Output Data Structure.xlsx', index=False)